# Digital
---

1. Data Preparation
2. Model Comparison
- K-mean
- LDA
- NMF
- Top2Vec > still in progress and will continue if time permits
- BERTopic > give up BERTopic because of time constraints

## 0. Import Libraries

In [1]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans


# Import NLP Libraries
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from tqdm import tqdm 
import pyLDAvis
import pyLDAvis.gensim_models
from top2vec import Top2Vec


# Set default Thai font
mpl.font_manager.fontManager.addfont('./THSarabunNew/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## 1. Data Preparation

### 1.1 Topic Selection

In [2]:
bkkbiz = pd.read_json('../datasets/bkkbiz_digital_processed.json')
matichon = pd.read_json('../datasets/matichon_digital_processed.json')

In [3]:
bkkbiz.head()

,ID,date,category,title,description,article,url,title_count,title_tokenize,description_count,description_tokenize,article_count,article_tokenize
0,620015,2014-11-27 10:08:00,การเงิน-การลงทุน,เนชั่นจ่อปรับค่าโฆษณา20%,"""เนชั่น มัลติมีเดีย กรุ๊ป""หวังรายได้ปีหน้าแตะ3...",นางสาวดวงกมล โชตะนา กรรมการผู้อำนวยการ บริษัท ...,https://www.bangkokbiznews.com/news/detail/620015,6,"[เนชั่น, จ่อ, ค่าโฆษณา]",25,"[เนชั่น, มัลติมีเดีย, กรุ๊ป, หวัง, รายได้, ปีห...",738,"[ดวงกมล, โช, ตะ, นา, กรรมการ, ผู้อำนวยการ, บริ..."
1,620023,2014-11-27 10:26:00,การเมือง,ทุ่ม80ล้านเปิดเวที77จว.,สปช.ลุยฟังความเห็นปฏิรูป-คิกออฟ1ธ.ค. รวบ2ผู้ต้...,สปช.ทุ่มงบ 80 ล้าน เปิด 1.3 พันเวทีปฏิรูป รับฟ...,https://www.bangkokbiznews.com/news/detail/620023,7,"[ทุ่ม, ล้าน, เวที, จว.]",22,"[สปช., ลุย, ฟัง, ความเห็น, ปฏิรูป, คิก, ออฟ, ธ...",1381,"[สปช., ทุ่ม, งบ, ล้าน, พัน, เวที, ปฏิรูป, รับฟ..."
2,620025,2014-11-27 10:35:00,ธุรกิจ,NOW26ปักหลักสยามสแควร์ เปิดสตูดิโอใหม่พันตรม.-...,เปิดแผนปี 2558 ดิจิทัลทีวี NOW26 เตรียมเปิดสตู...,เพิ่มช่องทางเข้าถึงคนรุ่นใหม่ สร้างประสบการณ์ร...,https://www.bangkokbiznews.com/news/detail/620025,13,"[NOW, ปักหลัก, สยามสแควร์, สตูดิโอ, พัน, ตรม, ...",21,"[แผน, ปี, ดิจิทัล, ทีวี, NOW, เตรียม, สตูดิโอ,...",631,"[ช่องทาง, เข้าถึง, คนรุ่นใหม่, สร้าง, ประสบการ..."
3,620054,2014-11-27 11:08:00,การเงิน-การลงทุน,NBC - ซื้อ,เตรียมปรับเพิ่มค่าโฆษณาช่อง Nation TV ขึ้น 30-50%,ปรับเพิ่มมูลค่าเหมาะสมเป็น 6 บาท เราปรับเพิ่มม...,https://www.bangkokbiznews.com/news/detail/620054,3,"[NBC, ซื้อ]",12,"[เตรียม, ค่าโฆษณา, ช่อง, Nation, TV]",657,"[มูลค่า, เหมาะสม, บาท, มูลค่า, เหมาะสม, หุ้น, ..."
4,620065,2014-11-27 11:37:00,ธุรกิจ,'ซิคเว่'เสนอแผน'ดิจิทัลอีโคโนมี',"""ซิคเว่"" เล็งผนึก ""เอไอเอส ทรู"" พร้อมควง ""ทีโอ...","หวังสานฝันนโยบายรัฐสัมฤทธิผล รอนัดวันเข้าพบ ""ห...",https://www.bangkokbiznews.com/news/detail/620065,12,"[ซิ, คเว่', เสนอ, แผน, ดิจิทัล, อี, โค, โน]",28,"[ซิ, คเว่"", เล็ง, ผนึก, เอไอเอส, ทรู, ควง, ทีโ...",1329,"[หวัง, สาน, ฝัน, นโยบาย, รัฐ, สัมฤทธิผล, รอ, น..."


In [4]:
matichon.head()

,title,article,date,category,url,id,title_count,title_tokenize,article_count,article_tokenize
0,"แห่ชมฝนดาวตก ""เจมินิดส์""คึกคัก ""ดอยอินทนนท์"" ช...",วันที่ 15 ธันวาคม นายศรันย์ โปษยะจินดา รองผู้อ...,2015-12-15 02:36:22,region,https://www.matichon.co.th/region/news_622,622,20,"[แห่, ชม, ฝนดาวตก, เจ, ส์"", คึกคัก, ดอย, อินทน...",721,"[วันที่, ธันวาคม, นา, ยศ, รัน, ย์, โป, ษ, ยะ, ..."
1,สมาชิกรุมจวกกม.คลื่นความถี่! 'วิทยา' ซัดสปท.ไม...,ถก ร่างแก้ไข พ.ร.บ.องค์กรจัดสรรคลื่นความถี่ ทำ...,2016-01-18 23:48:28,politics,https://www.matichon.co.th/politics/news_5384,5384,19,"[สมาชิก, รุม, จวก, กม., คลื่นความถี่, วิทยา, ซ...",558,"[ถก, ร่าง, พ.ร.บ., องค์กร, จัดสรร, คลื่นความถี..."
2,โครงการ 3 ประสานความร่วมมือ กสทช. กสท และ CABSAT,โครงการ 3 ประสานความร่วมมือ กสทช. กสท และ CABS...,2016-01-20 09:08:59,publicize,https://www.matichon.co.th/publicize/news_6608,6608,8,"[โครงการ, ประสาน, ความร่วมมือ, กสทช., กสท, CAB...",425,"[โครงการ, ประสาน, ความร่วมมือ, กสทช., กสท, CAB..."
3,"เอไอเอส จัดเต็ม มหกรรม ""AIS 4G ADVANCED NEW WO...","เอไอเอสขอเชิญลูกค้าและประชาชน ร่วมงานมหกรรม ""A...",2016-01-24 09:08:54,publicize,https://www.matichon.co.th/publicize/news_11306,11306,14,"[เอไอเอส, เต็ม, มหกรรม, AIS, G, ADVANCED, NEW,...",475,"[เอไอเอส, เชิญ, ลูกค้า, ประชาชน, ร่วมงาน, มหกร..."
4,"CAT เปิดศูนย์นวัตกรรม ""IOT City Innovation Ce...",บมจ. กสท โทรคมนาคม (CAT) วางโครงข่ายระบบไอทีค...,2016-01-24 12:46:00,publicize,https://www.matichon.co.th/publicize/news_11463,11463,10,"[CAT, ศูนย์, นวัตกรรม, IOT, City, Innovation, ...",404,"[บมจ., กสท, โทรคมนาคม, CAT, วาง, โครงข่าย, ระบ..."


### 1.2 Bag-of-Words

In [5]:
# Finction to store n_word in dict
def featurize(token_list):
    token_list=token_list
    features = {}
    for token in token_list:
        features[token]=1
    return features

In [7]:
bkkbiz_token = bkkbiz['article_tokenize'].apply(featurize)
matichon_token = matichon['article_tokenize'].apply(featurize)

In [8]:
vec_bkkbiz = DictVectorizer(sparse=True)
bkkbiz_vec = vec_bkkbiz.fit_transform(bkkbiz_token)

vec_matichon = DictVectorizer(sparse=True)
matichon_vec = vec_matichon.fit_transform(matichon_token)

In [9]:
bkkbiz.shape, matichon.shape

((16464, 13), (10433, 10))

In [10]:
bkkbiz_vec, matichon_vec

(<16464x63496 sparse matrix of type '<class 'numpy.float64'>'
 	with 2760975 stored elements in Compressed Sparse Row format>,
 <10433x59082 sparse matrix of type '<class 'numpy.float64'>'
 	with 2135687 stored elements in Compressed Sparse Row format>)

## 2. Model Comparison

### 2.1 K-Means

In [11]:
#def kmeans_topic(data_vec=None, n_clusters=10):
#    kmeans = KMeans(n_clusters=n_clusters, 
#                    init='k-means++', 
#                    max_iter=100, 
#                    n_init=1, 
#                    random_state=42)
#
#    kmeans.fit(data_vec)

#    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    
#    if data_vec == bkkbiz_vec:
#        terms = vec_bkkbiz.get_feature_names_out()
#    elif data_vec == matichon_vec:
#        terms = vec_matichon.get_feature_names_out()
#    else:
#        print('error')

#    for i in range(n_clusters):
#        print(f'Topic {i+1}')
#        print('-'*10)
#        for i in order_centroids[i,:10]:
#            print(terms[i])
#        print('\n')

In [12]:
# kmeans_topic(data_vec=bkkbiz_vec, n_clusters=10)

In [13]:
kmeans = KMeans(n_clusters=10, 
                init='k-means++', 
                max_iter=100, 
                n_init=1, 
                random_state=42)

In [14]:
kmeans.fit(bkkbiz_vec)

KMeans(max_iter=100, n_clusters=10, n_init=1, random_state=42)

In [15]:
# KMeans: Bkkbiz
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vec_bkkbiz.get_feature_names_out()
print('Bkkbiz')
print('K-Means')
print('*'*20)
for i in range(10):
    print(f'Topic {i+1}')
    print('-'*10)
    words = []
    for i in order_centroids[i,:15]:
        words.append(terms[i])
    print(words)
    print(' ')

Bkkbiz
K-Means
********************
Topic 1
----------
['ดิจิทัล', 'ทักษะ', 'ความร่วมมือ', 'แพลตฟอร์ม', 'พันธุ์', 'บุคลากร', 'ขับเคลื่อน', 'บริษัท', '์', 'วัฒน', 'คน', 'การพัฒนา', 'นวัตกรรม', 'คลาวด์', 'ไมโครซอฟท์']
 
Topic 2
----------
['เนจ', 'BNK', 'บริษัท', 'สร้าง', 'นท', 'อิน', 'การพัฒนา', 'วงการ', 'จำกัด', 'iAM', 'เม้นท์', 'จับมือ', 'เด้', '์', 'เพ']
 
Topic 3
----------
['ดิจิทัล', 'คาด', 'ปี', 'ล้าน', 'บาท', 'หุ้น', 'ราคา', 'ซื้อ', 'ไทย', 'ตลาด', 'เดือน', 'เพิ่มขึ้น', 'ธุรกิจ', 'ลดลง', 'บวก']
 
Topic 4
----------
['ดิจิทัล', 'ปี', 'ไทย', 'ธุรกิจ', 'สร้าง', 'บริษัท', 'ทำ', 'สำหรับ', 'ประเทศ', 'มากขึ้น', 'ล้าน', 'เติบโต', 'ที่จะ', 'ดี', 'ตลาด']
 
Topic 5
----------
['หมุด', 'ร้าน', "เว่ย'", 'ลูก', 'ซ์', 'บาร์เลย์', '์', 'ส', 'แอนด์', 'ธุรกิจ', 'ร', 'ปฏิรูป', 'พายุ', 'ไทย', 'คลาวด์']
 
Topic 6
----------
['ดิจิทัล', 'ไทย', 'ประเทศ', 'ปี', 'สร้าง', 'ที่จะ', 'สำหรับ', 'เรื่อง', 'ระบบ', 'พัฒนา', 'ทำ', 'ดี', 'กล่าวว่า', 'การพัฒนา', 'เทคโนโลยี']
 
Topic 7
----------
['บาร์เลย์', 'ธ.ค.'

In [16]:
# KMeans: Matichon
kmeans = KMeans(n_clusters=10, 
                init='k-means++', 
                max_iter=100, 
                n_init=1, 
                random_state=42)

In [17]:
# KMeans: Matichon
kmeans.fit(matichon_vec)

KMeans(max_iter=100, n_clusters=10, n_init=1, random_state=42)

In [19]:
# KMeans: Matichon
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vec_matichon.get_feature_names_out()
print('MATICHON')
print('K-Means')
print('*'*20)
for i in range(10):
    print(f'Topic {i+1}')
    print('-'*10)
    words = []
    for i in order_centroids[i,:15]:
        words.append(terms[i])
    print(words)
    print(' ')

MATICHON
K-Means
********************
Topic 1
----------
['ดิจิทัล', 'ไทย', 'สำหรับ', 'ปี', 'บริษัท', 'ประเทศ', 'เทคโนโลยี', 'สร้าง', 'จำกัด', 'คน', 'ดี', 'ยุค', 'ทำ', 'ธุรกิจ', 'ที่จะ']
 
Topic 2
----------
['คน', 'เรื่อง', 'ไทย', 'วันที่', 'ประเทศ', 'ดิจิทัล', 'ประชาชน', 'รัฐบาล', 'ปี', 'ทำ', 'อ.', 'พล', 'ดี', 'ที่จะ', 'จันทร์']
 
Topic 3
----------
['มหาวิทยาลัย', 'การทำสัญญา', 'ที่อยู่', 'ติดต่อ', 'ยอดขาย', 'อนัญ', 'ไทยใหญ่', 'ไทยแลนด์', 'ศาสตราจารย์', 'ไทย', 'อนาคต', 'ตารางเมตร', 'บางส่วน', 'บางคน', 'ยังอยู่']
 
Topic 4
----------
['โทรคมนาคม', 'กิจการ', 'แห่งชาติ', 'คณะกรรมการ', 'กสทช.', 'โทรทัศน์', 'กระจายเสียง', 'ดิจิทัล', 'วันที่', 'กล่าวว่า', 'สำนักงาน', 'ดี', 'บริษัท', 'เรื่อง', 'ไทย']
 
Topic 5
----------
['ถ่ายทอดสด', 'น.', 'รับสัญญาณ', 'ดิจิทัล', 'โทรทัศน์', 'ดี', 'พรีเมียร์', 'อันดับ', 'สัปดาห์', 'ออนไลน์', 'กล่อง', 'เอช', 'ตัว', 'เวลา', 'com']
 
Topic 6
----------
['วันที่', 'ดิจิทัล', 'เรื่อง', 'ประชาชน', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม', 'ดี', 'รัฐมนตรี', 'ข้อมูล

In [ ]:
# Create Scatterplot!

# Figsize
#plt.figure(figsize=(10,8))

# Map colors for different clusters
#colors = ['red', 'green', 'blue']
#df['color'] = df['cluster'].map(lambda p: colors[p]) # 0=red, 1=green, 2=blue 

# Plot points
#ax = df.plot(kind='scatter', x='x1', y='x2', figsize=(10,8));

# Plot Centroids
#centroids.plot(kind='scatter', x='x1', y='x2', marker='*',
             c = ['r', 'g', 'b'], s=500, ax=ax); # s == size

#plt.title('Converged Clusters', size=20);

### 2.2 LDA

In [20]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    perplexity = model.log_perplexity(corpus)
    #topic = pd.DataFrame(model.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
    print(f'LDA Model with {num_topics} topics')
    print(f'Perplexity: {perplexity.round(2)}')
    print('-'*60)
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=15)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model

In [21]:
# Visualization
def lda_vis(data=None, num_topics=20):
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return viz

In [22]:
bkkbiz_lda = lda_model(data=bkkbiz['article_tokenize'], num_topics=10)

LDA Model with 10 topics
Perplexity: -8.22
------------------------------------------------------------
Topic 1
['ธุรกิจ', 'บริษัท', 'ดิจิทัล', 'ลูกค้า', 'ไทย', 'ปี', 'บริการ', 'เทคโนโลยี', 'จำกัด', 'สร้าง', 'องค์กร', 'เติบโต', 'ล้าน', 'ธนาคาร', 'ระบบ']
------------------------------------------------------------
Topic 2
['ล้าน', 'ปี', 'บาท', 'ออนไลน์', 'สินค้า', 'ดิจิทัล', 'ธุรกิจ', 'เติบโต', 'สื่อ', 'ไทย', 'ผู้บริโภค', 'บริษัท', 'แบรนด์', 'แพลตฟอร์ม', 'ตลาด']
------------------------------------------------------------
Topic 3
['บาท', 'ล้าน', 'ปี', 'ราคา', 'หุ้น', 'ดิจิทัล', 'สลาก', 'สินทรัพย์', 'เงิน', 'คาด', 'ไทย', 'ตลาด', 'เดือน', 'ลดลง', 'เพิ่มขึ้น']
------------------------------------------------------------
Topic 4
['ไทย', 'เศรษฐกิจ', 'ประเทศ', 'การลงทุน', 'อุตสาหกรรม', 'จีน', 'อาเซียน', 'ปี', 'ส่งเสริม', 'การค้า', 'โครงการ', 'อีซี', 'พื้นที่', 'ล้าน', 'ตะวันออก']
------------------------------------------------------------
Topic 5
['คน', 'เด็ก', 'บ้าน', 'ดิจิทัล', 'โควิด', 'ว

In [23]:
lda_vis(data=bkkbiz['article_tokenize'], num_topics=10)

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.001124  0.028734       1        1  26.233496
3      0.023690  0.119183       2        1  16.194992
6      0.141514  0.041573       3        1  13.830270
2     -0.152290  0.049853       4        1   9.955040
1     -0.092145  0.090081       5        1   9.267866
9      0.136165 -0.129786       6        1   8.898198
0      0.015690  0.062971       7        1   6.705571
7      0.062925 -0.035022       8        1   4.826344
4      0.073057 -0.076702       9        1   2.289659
8     -0.209730 -0.150885      10        1   1.798564, topic_info=           Term          Freq         Total Category  logprob  loglift
47      ดิจิทัล  54917.000000  54917.000000  Default  30.0000  30.0000
759        เงิน  10746.000000  10746.000000  Default  29.0000  29.0000
77          บาท  29628.000000  29628.000000  Default  28.0000  28.0000
751   สินทรัพย์   7322.000000   7322.000000  Default  27.0000  27.0000
120        ล้าน  32542.000000  32542.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
411      รายงาน    532.367225   5732.630379  Topic10  -5.2403   1.6416
120        ล้าน    694.189122  32542.817580  Topic10  -4.9748   0.1706
110       ระดับ    552.133587  10377.406632  Topic10  -5.2038   1.0846
2076      สหรัฐ    466.441084   4506.859594  Topic10  -5.3725   1.7500
60           ทำ    571.179245  22457.349869  Topic10  -5.1699   0.3465

[895 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
1989       1  0.003710   ADVANC
1989       2  0.001237   ADVANC
1989       4  0.870511   ADVANC
1989       5  0.114996   ADVANC
1989       6  0.001237   ADVANC
...      ...       ...      ...
999       10  0.014184        ์
12986      1  0.010885  ์เบิร์ก
12986      3  0.021771  ์เบิร์ก
12986      8  0.881720  ์เบิร์ก
12986     10  0.076198  ์เบิร์ก

[4596 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 7, 3, 2, 10, 1, 8, 5, 9])

In [24]:
matichon_lda = lda_model(data=matichon['article_tokenize'], num_topics=10)

LDA Model with 10 topics
Perplexity: -8.34
------------------------------------------------------------
Topic 1
['คน', 'เรื่อง', 'ประเทศ', 'การศึกษา', 'ประชาชน', 'ไทย', 'ครู', 'รัฐบาล', 'ที่จะ', 'สังคม', 'ระบบ', 'ทำ', 'พัฒนา', 'ข้อมูล', 'ปี']
------------------------------------------------------------
Topic 2
['ไทย', 'ปี', 'เศรษฐกิจ', 'ประเทศ', 'การลงทุน', 'ล้าน', 'อาเซียน', 'ส่งเสริม', 'การค้า', 'อุตสาหกรรม', 'จีน', 'บาท', 'โครงการ', 'ภูมิภาค', 'การพัฒนา']
------------------------------------------------------------
Topic 3
['บาท', 'ล้าน', 'กิจการ', 'กสทช.', 'ปี', 'บริษัท', 'จำนวน', 'คลื่นความถี่', 'โทรคมนาคม', 'ดิจิทัล', 'วันที่', 'จำกัด', 'เรื่อง', 'ไทย', 'เดือน']
------------------------------------------------------------
Topic 4
['ไทย', 'ปี', 'คน', 'โลก', 'ประเทศ', 'สร้าง', 'ทำ', 'เรื่อง', 'ดิจิทัล', 'แบรนด์', 'ดี', 'บริษัท', 'สำหรับ', 'ที่จะ', 'เทคโนโลยี']
------------------------------------------------------------
Topic 5
['โควิด', 'คน', 'ออนไลน์', 'โรค', 'สุขภาพ', 'โรงพยาบาล

In [25]:
lda_vis(data=matichon['article_tokenize'], num_topics=10)

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.053221  0.006954       1        1  16.610384
8      0.121600 -0.001979       2        1  15.158204
3      0.098699  0.015641       3        1  14.252280
5     -0.011306 -0.113257       4        1  14.215791
6      0.059726 -0.100281       5        1   9.039570
0      0.104701  0.130939       6        1   8.330723
9     -0.112654 -0.000128       7        1   7.532086
1     -0.048133 -0.081690       8        1   7.057145
4     -0.028771  0.048164       9        1   4.062343
7     -0.130640  0.095638      10        1   3.741475, topic_info=            Term          Freq         Total Category  logprob  loglift
653       ข้อมูล  15895.000000  15895.000000  Default  30.0000  30.0000
2346         ครู   4934.000000   4934.000000  Default  29.0000  29.0000
1667    การศึกษา   6799.000000   6799.000000  Default  28.0000  28.0000
556          บาท  12926.000000  12926.000000  Default  27.0000  27.0000
97       ประชาชน  12581.000000  12581.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
353       เรื่อง    471.995002  15454.755160  Topic10  -5.6960  -0.2030
653       ข้อมูล    452.450753  15895.140883  Topic10  -5.7383  -0.2734
97       ประชาชน    418.666129  12581.564353  Topic10  -5.8159  -0.1172
619          ไทย    418.574797  36798.813976  Topic10  -5.8161  -1.1907
10392  แพร่ระบาด    381.141063   4324.054778  Topic10  -5.9098   0.8569

[944 rows x 6 columns], token_table=       Topic      Freq Term
term                       
6964       1  0.007074   AI
6964       2  0.829083   AI
6964       3  0.032541   AI
6964       4  0.002830   AI
6964       5  0.064374   AI
...      ...       ...  ...
18104      6  0.070492   …
18104      8  0.869401   …
18104     10  0.023497   …
31325      1  0.013311    ⦁
31325      4  0.985029    ⦁

[4707 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 4, 6, 7, 1, 10, 2, 5, 8])

### 2.3 NMF

In [ ]:
#data = econ_b['article_tokenize'].apply(lambda x:' '.join(x))
#cvec = CountVectorizer(token_pattern= "\b[A-zก-๙][A-z\.\-ก-๙]*\b")
#data = cvec.fit_transform(data)

In [34]:
def nmf(data=None, n_components=10, vectorizer=None):
    model = NMF(n_components=n_components)
    model.fit(data)
    features = model.transform(data)
    components = pd.DataFrame(model.components_,
                                columns=vectorizer.get_feature_names())
    # Topic & Words
    
    for i in range(n_components):
        words = []
        topic = components.iloc[i]
        print(f'Top 15 words for topic {i+1} are:')
        for n, word in enumerate(topic.nlargest(15)):
            words.append(topic.nlargest(20).keys()[n])
        print(words)
        print('\n')

In [35]:
nmf(data=bkkbiz_vec, n_components=10, vectorizer=vec_bkkbiz)

Top 15 words for topic 1 are:
['คน', 'เรื่อง', 'ตัวเอง', 'ทำ', 'ดู', 'รู้', 'ตัว', 'ดี', 'เวลา', 'ทำงาน', 'หา', 'เหมือน', 'บอ', 'เกิดขึ้น', 'ต้องการ']


Top 15 words for topic 2 are:
['วันที่', 'ประชาชน', 'คณะกรรมการ', 'เรื่อง', 'ดำเนินการ', 'แห่งชาติ', 'รัฐบาล', 'รัฐมนตรี', 'อ.', 'ดี', 'กล่าวว่า', 'พล', 'พิจารณา', 'ว่าการ', 'กรณี']


Top 15 words for topic 3 are:
['คาด', 'ราคา', 'บาท', 'หุ้น', 'ลดลง', 'เพิ่มขึ้น', 'ล้าน', 'ตลาด', 'ซื้อ', 'เดือน', 'ปี', 'บวก', 'ปัจจัย', 'การลงทุน', 'แนวโน้ม']


Top 15 words for topic 4 are:
['พายุ', 'ส', 'ร', 'หัว', 'ชู', 'ซ์', 'แอนด์', 'ลูก', 'ปัก', 'สไตล์', 'หมุด', 'ร้าน', 'คลาวด์', 'ปฏิรูป', 'เลา']


Top 15 words for topic 5 are:
['ส่วนบุคคล', 'ข้อมูล', 'ออนไลน์', 'ระบบ', 'คุ้มครอง', 'อาเซียน', 'มิติ', 'ท้าทาย', 'งานแสดงสินค้า', 'ดิส', 'คนรุ่นใหม่', "รัปชั่น'", 'ดิจิทัล', 'ปี', 'คน']


Top 15 words for topic 6 are:
['โลก', 'โอ', 'เงิน', 'เตือน', 'แข่งขัน', 'พัน', 'อี', 'จีดีพี', 'ภาษี', 'ศักยภาพ', 'กองทุน', 'ล้าน', 'ตกลง', 'พ่อ', 'เรือ']


Top 15 wo

In [36]:
nmf(data=matichon_vec, n_components=10, vectorizer=vec_matichon)

Top 15 words for topic 1 are:
['เทคโนโลยี', 'องค์กร', 'ธุรกิจ', 'บริษัท', 'พัฒนา', 'นวัตกรรม', 'บริการ', 'สร้าง', 'การพัฒนา', 'ระบบ', 'ข้อมูล', 'ที่จะ', 'การทำงาน', 'มีประสิทธิภาพ', 'ก้าว']


Top 15 words for topic 2 are:
['วันที่', 'รัฐมนตรี', 'ว่าการ', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม', 'อ.', 'นายกรัฐมนตรี', 'พล', 'ประชาชน', 'ประยุทธ์', 'จันทร์', 'โอชา', 'กระทรวง', 'ดี', 'เรื่อง', 'อี']


Top 15 words for topic 3 are:
['คน', 'ทำ', 'เรื่อง', 'เกิดขึ้น', 'รู้', 'ตัวเอง', 'เป็นเรื่อง', 'ดู', 'ออกมา', 'โลก', 'ที่จะ', 'เข้ามา', 'เหมือน', 'ตัว', 'บอ']


Top 15 words for topic 4 are:
['ล้าน', 'ดิจิทัล', 'ลูกค้า', 'ปี', 'ออนไลน์', 'บาท', 'ไทย', 'ธุรกิจ', 'กล่าวว่า', 'ตลาด', 'สินค้า', 'บริษัท', 'ช่องทาง', 'บริการ', 'ซื้อ']


Top 15 words for topic 5 are:
['์', 'ชัย', 'ประธาน', 'บริษัท', 'จำกัด', 'ริ', 'งาน', 'ไทย', 'สุ', 'กุล', 'ดิจิทัล', 'ศรี', 'ศักดิ์', 'สม', 'รอง']


Top 15 words for topic 6 are:
['เศรษฐกิจ', 'การลงทุน', 'ส่งเสริม', 'ประเทศ', 'อุตสาหกรรม', 'ไทย', 'การค้า', 'นโยบาย', 'ก

### 2.4 Top2Vec
- fast learn, learn and deep learn

In [ ]:
# min_count =
# ngram_vocab =
# embedding_model= (doc2vec)
# workers = cpu amount

In [29]:
# Create function that tokenize, remove stopwords and perform RegularExpression
def tokenize(txt):
  
    ''' 
    Description:
        A function that returns tokenized words with stop words and other regular expressions 
        that do not match English and Thai alphabets removed

    Parameters:
        txt = text string

    Return:
        word tokens as list
    '''

    stopwords = thai_stopwords()
    tokens = word_tokenize(txt, keep_whitespace = False)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-\uDe5c]+', token)]
    return tokens

In [101]:
# speed='fast-learn'
def top2vec(documents=None, n_topics=None, speed='fast-learn', ngram_vocab=None):
    top2vec = Top2Vec(documents=documents,
                      speed=speed,
                      tokenizer=tokenize)
    topic_words, word_scores, topic_nums = top2vec.get_topics()
    print(f'Topic: {topic_nums+1}')
    print(f'Words: {topic_words}')
    return top2vec, topic_nums, topic_words

In [87]:
bkkbiz_list = bkkbiz['article'].tolist()
matichon_list = matichon['article'].tolist()

In [105]:
top2vec = Top2Vec(documents=bkkbiz_list,speed='fast-learn',tokenizer=tokenize)
topic_words, word_scores, topic_nums = top2vec.get_topics()

2022-11-16 21:07:13,604 - top2vec - INFO - Pre-processing documents for training
2022-11-16 21:09:17,960 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 21:18:40,295 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 21:18:46,577 - top2vec - INFO - Finding dense areas of documents
2022-11-16 21:18:49,807 - top2vec - INFO - Finding topics


In [143]:
topic_words[:20]

array([['กสทช.', 'เมกะเฮิรตซ์', 'โทรคมนาคม', 'คลื่นความถี่',
        'กระจายเสียง', 'โทรทัศน์', 'เฮิรตซ์', 'ประมูล', 'กสท', 'ฐากร',
        'กิจการ', 'การประมูล', 'ตัณฑ', 'คลื่น', 'กิกะ', 'ใบอนุญาต',
        'ทีโอที', 'โครงข่าย', 'คณะกรรมการ', 'บอร์ด', 'สัมปทาน', 'MHz',
        'ทค.', 'ประชาพิจารณ์', 'ความถี่', 'แห่งชาติ', 'ย่าน', 'เรียกคืน',
        'มีมติ', 'บมจ.', 'bet', 'กระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม',
        'ที่ประชุม', 'ไมโครโฟน', 'กสท.', 'คณะรัฐมนตรี', 'คม',
        'เลขาธิการ', 'โทรศัพท์เคลื่อนที่', 'ดำเนินการ', 'หลักเกณฑ์',
        'เสนอ', 'รัฐวิสาหกิจ', 'โอเปอเรเตอร์', 'อสมท', 'เยียวยา',
        'วิทยุกระจายเสียง', 'เฮิรตช์', 'วิทยุโทรทัศน์', 'อนุกรรมการ'],
       ['องค์กร', 'ทรานส์', 'ฟอ', 'ไอที', 'ไอ', 'คอมพิวติ้ง', 'ออราเคิล',
        'โซลูชั่น', 'สฟอร์', 'เทคโนโลยี', 'ร์', 'เคียว', 'ไฮบริด',
        'การเปลี่ยนแปลง', 'เอ', 'การทำงาน', 'ค็อก', '์ท', 'ธิงส์(',
        'ชั่น', 'คลาวด์', 'ความยืดหยุ่น', 'ปรับเปลี่ยน', 'ดจ์', 'ธุรกิจ',
        'ซับซ้อน', 'ทิฟ', 'ไอบีเอ

In [146]:
documents, document_scores, document_ids = top2vec.search_documents_by_topic(topic_num=3, 
                                                                             num_docs=20)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f'Document: {doc_id}, Score: {score}')
    print('-'*20)
    print(doc)
    print('-'*20)
    print()

Document: 15205, Score: 0.7950366735458374
--------------------
นางปรัศนี อุยามะพันธุ์ ผู้ช่วยผู้จัดการใหญ่ ธนาคารกรุงเทพ จำกัด (มหาชน) BBL กล่าวว่า แผนยุทธ์ศาสตร์ของธนาคารในปีนี้ คือการเดินหน้าสู่การเป็น "ธนาคารดิจิทัล" โดยการยกระดับการให้บริการบนดิจิทัล ผ่านโมบายแบงกิง เพื่อให้ตอบโจทย์ลูกค้าบนยุคดิจิทัลมากขึ้น

ล่าสุด ธนาคารมีแผนเปิดให้บริการสินเชื่อผ่านดิจิทัลผ่านโมบายแบงกิง ทั้งดิจิทัลเลนดิ้ง สินเชื่อบุคคล บัตรเครดิต สินเชื่อบ้าน และสินเชื่อสำหรับผู้ประกอบการขนาดเล็ก ที่สามารถสมัครขอสินเชื่อได้ผ่านแอป
--------------------

Document: 15502, Score: 0.7624091506004333
--------------------
นายนริศ อารักษ์สกุลวงศ์ หัวหน้าทีมดิจิทัล ทีทีบี สปาร์ค เปิดเผยว่า ธนาคารได้จัดตั้งทีม "ttb spark" ซึ่งจะถือเป็นจุดเปลี่ยนสำคัญด้าน Digital transformation ของธนาคาร เพื่อเสริมแกร่งศักยภาพ และเพิ่มขีดความสามารถด้านดิจิทัลของธนาคารให้สอดคล้องกับแนวคิด "Humanized Digital Banking" หรือ ดิจิทัลแบงก์กิ้งที่เป็นมิตร และรู้ใจ ใช้ง่าย แก้ปัญหาของลูกค้า และช่วยกระดับการให้บริการลูกค้าได้หลากหลาย ครอบคลุมทุกควา

In [112]:
top2vec_2 = Top2Vec(documents=matichon_list,speed='fast-learn',tokenizer=tokenize)
topic_words_2, word_scores_2, topic_nums_2 = top2vec_2.get_topics()

2022-11-16 21:58:40,660 - top2vec - INFO - Pre-processing documents for training
2022-11-16 21:59:49,855 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 22:05:19,899 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 22:05:24,032 - top2vec - INFO - Finding dense areas of documents
2022-11-16 22:05:24,300 - top2vec - INFO - Finding topics


In [137]:
topic_words_2[0:20]

array([['คลื่นความถี่', 'เมกะเฮิรตซ์', 'กสทช.', 'การประมูล', 'ประมูล',
        'เฮิรตซ์', 'กระจายเสียง', 'โทรทัศน์', 'กิจการ', 'ซ์)',
        'โอเปอเรเตอร์', 'ใบอนุญาต', 'ตัณฑ', 'โทรคมนาคม', 'ฐากร', 'ย่าน',
        'คลื่น', 'เรียกคืน', 'กิกะ', 'ทีวี', 'โครงข่าย', 'งวด', 'G',
        'ความถี่', 'MHz', 'ภาคพื้นดิน', 'สปริง', 'โทรศัพท์เคลื่อนที่',
        'จัดสรร', 'อสมท', 'ค่าชดเชย', 'ชำระ', 'ฐา', 'หลักเกณฑ์',
        'สัมปทาน', 'ร์เลส', 'ผู้ให้บริการ', 'ช่อง', 'เงินชดเชย',
        'ภาษีมูลค่าเพิ่ม', 'เชาวน์', 'การออกอากาศ', 'ค่าเช่า',
        'วิทยุโทรทัศน์', 'คอมมิวนิเคชั่น', 'บรอดแบนด์', 'การอนุญาต',
        'GHz', 'วิทยุกระจายเสียง', 'ค่า'],
       ['ดำเนินคดี', 'กระทำความผิด', 'ความผิด', 'คำสั่งศาล', 'แจ้งความ',
        'เข้าข่าย', 'การกระทำ', 'ปราบปราม', 'พ.ต.อ.', 'จับกุม',
        'บิดเบือน', 'พนักงานสอบสวน', 'ผิดกฎหมาย', 'พยานหลักฐาน',
        'บังคับการ', 'ท.', 'ท.)', 'การจับกุม', 'สำนักงานตำรวจแห่งชาติ',
        'โพสต์', 'เท็จ', 'ประมวลกฎหมายอาญา', 'ข้อความ',
        'เจ้าหน้าท

In [139]:
documents, document_scores, document_ids = top2vec_2.search_documents_by_topic(topic_num=3, 
                                                                             num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f'Document: {doc_id}, Score: {score}')
    print('-'*20)
    print(doc)
    print('-'*20)
    print()

Document: 3662, Score: 0.7642788887023926
--------------------
นายพิเชฐ ดุรงคเวโรจน์ รัฐมนตรีว่าการกระทรวงดิจิทัลเพื่อเศรษฐกิจและสังคม (ดีอี) เป็นสักขีพยานในพิธีลงนามบันทึกข้อตกลงความร่วมมือว่าด้วยการส่งเสริมและสนับสนุนการประยุกต์ใช้เทคโนโลยีและนวัตกรรมดิจิทัล ระหว่างสำนักงานส่งเสริมเศรษฐกิจดิจิทัล (ดีป้า) กับ บริษัท ไปรษณีย์ไทย จำกัด (ปณท)  2 หน่วยงานในสังกัดกระทรวงดีอี ณ อาคารชาเลนเจอร์ ฮอลล์ 1-3 ศูนย์แสดงสินค้าและการประชุมอิมแพ็ค เมืองทองธานี โดยความร่วมมือดังกล่าว เพื่อส่งเสริมการนำเทคโนโลยีและนวัตกรรมดิจิทัลมาพัฒนาชุมชนให้มีขีดความสามารถ เพื่อเพิ่มประสิทธิภาพในการบริหารจัดการองค์กรในชุมชน รวมถึงการเชื่อมโยงการเข้าถึงเทคโนโลยีและนวัตกรรมดิจิทัลอื่นๆ ที่เกี่ยวข้องกับความต้องการของกลุ่มองค์กรในชุมชน
นอกจากนี้ ยังเป็นการพัฒนาบุคลากร และผู้ประกอบการในชุมชนกับการใช้ประโยชน์จากเทคโนโลยีและนวัตกรรมดิจิทัลด้านการบริหารจัดการองค์กร การพัฒนาช่องทางการจัดจำหน่ายผลิตภัณฑ์และบริการของชุมชน รวมถึงการเพิ่มคุณค่าให้กับผลิตภัณฑ์และบริการของชุมชน ตลอดจนเพิ่มโอกาสให้ชุมชนสามารถเข้าถึงนโยบายในการส่งเส

In [102]:
top2vec(documents=bkkbiz_list, speed='fast-learn')

2022-11-16 20:53:52,678 - top2vec - INFO - Pre-processing documents for training
2022-11-16 20:56:01,010 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 21:05:16,428 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 21:05:22,078 - top2vec - INFO - Finding dense areas of documents
2022-11-16 21:05:24,513 - top2vec - INFO - Finding topics


Topic: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144]
Words: [['กสทช.' 'เมกะเฮิรตซ์' 'กระจายเสียง' ... 'โทรศัพท์เคลื่อนที่' 'เห็นชอบ'
  'สิทธิ์']
 ['องค์กร' 'ไอที' 'ทรานส์' ... 'ไล' 'มากขึ้น' 'ความสามารถ']
 ['กระทำความผิด' 'ปลอม' 'ดำเนินคดี' ... 'ตรวจค้น' 'การสืบสวน' 'เอาผิด']
 ...
 ['bangkokbiznews' 'detail' 'news' ... 'เป็นการชั่วคราว' 'อักเสบ' 'พุธ']
 ['พลุ' 'สน.' 'พนักงานสอบสวน' ... 'ประกันตัว' 'ผบ.' 'ผบ']
 ['Futures' 'การซื้อขาย' 'Gold' ... 'บาร์เรล' 'F

(<top2vec.Top2Vec.Top2Vec at 0x7fbbc58588e0>,
 array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143]),
 array([['กสทช.', 'เมกะเฮิรตซ์', 'กระจายเสียง', ..., 'โทรศัพท์เคลื่อนที่',
         'เห็นชอบ', 'สิทธิ์'],
        ['องค์กร', 'ไอที', 'ทร

In [103]:
topic_words

array([['องค์กร', 'ไอที', 'คอมพิวติ้ง', 'คลาวด์', 'ซับซ้อน',
        'ความยืดหยุ่น', 'ออราเคิล', 'ฟอ', 'ไฮบริด', 'ไอ', 'ยืดหยุ่น',
        'ค็อก', 'ทรานส์', 'ไอบีเอ็ม', '์ท', 'โซลูชั่น', 'ดจ์', 'ทิฟ',
        'เอ', 'ปัญญาประดิษฐ์', 'การทำงาน', 'เลิร์น', 'ความคาดหวัง',
        'สฟอร์', 'ธิงส์(', 'ร์', 'เคียว', 'การประมวลผล', 'ตื่นตัว',
        'เดลล์', 'สภาพแวดล้อม', 'แปซิฟิก', 'ธิงส์', 'การเปลี่ยนแปลง',
        'ความสามารถ', 'เทคโนโลยี', 'ปรับเปลี่ยน', 'เป็นตัว', 'ควอนตัม',
        'โอที', 'คล่องตัว', 'ปฐ', 'แมชชีน', 'มัลติ', 'ไมโครซอฟท์',
        'ชาญฉลาด', 'การปฏิสัมพันธ์', 'ฟอร์ซ', 'กุญแจ', 'ผลลัพธ์'],
       ['ประธานาธิบดี', 'เกาหลีเหนือ', 'กรุง', 'วอชิงตัน', 'โอบามา',
        'ปูติน', 'ซีเรีย', 'ยูเอ็น', 'ผู้อพยพ', 'ผู้นำประเทศ', 'การทูต',
        'คิม', 'ผู้ต้องสงสัย', 'รัสเซีย', 'สันติภาพ', 'คว่ำบาตร',
        'ตุรกี', 'โดนัลด์', 'การก่อการร้าย', 'สิทธิมนุษยชน', 'โซล',
        'กองกำลัง', 'ทางทหาร', 'เรียกร้อง', 'สังหาร', 'สำนักข่าว',
        'ยูเครน', 'กระทรวงการต่างประเทศ', 'ซ

In [89]:
top2vec(documents=matichon_list, n_topics=10, speed='learn')

2022-11-15 16:03:56,875 - top2vec - INFO - Pre-processing documents for training
2022-11-15 16:05:08,286 - top2vec - INFO - Creating joint document/word embedding
2022-11-15 16:16:38,679 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-15 16:16:42,374 - top2vec - INFO - Finding dense areas of documents
2022-11-15 16:16:42,611 - top2vec - INFO - Finding topics


Topic: [ 1  2  3  4  5  6  7  8  9 10]
Words: [['รัฐประหาร' 'ประชาธิปไตย' 'ทางการเมือง' 'การเมือง' 'นักการเมือง'
  'การรัฐประหาร' 'ยึดอำนาจ' 'เผด็จการ' 'รัฐธรรมนูญ' 'การเลือกตั้ง'
  'อำนาจ' 'ม็อบ' 'ระบอบ' 'อุดมการณ์' 'ส.ว.' 'พรรคอนาคตใหม่'
  'พรรคการเมือง' 'เรียกร้อง' 'การสืบทอด' 'ร่างรัฐธรรมนูญ' 'ตรงกันข้าม'
  'ระบอบประชาธิปไตย' 'เลือกตั้ง' 'การเคลื่อนไหว' 'ผู้มีอำนาจ'
  'ความขัดแย้ง' 'ความชอบธรรม' 'ชุมนุม' 'บ้านเมือง' 'ประชามติ' 'ทักษิณ'
  'การต่อสู้' 'ธร' 'มวลชน' 'พรรค' 'พรรคเพื่อไทย' 'ขัดแย้ง' 'ชอบธรรม' 'กป'
  'ชนชั้นนำ' 'ความเชื่อ' 'ชินวัตร' 'ปลดแอก' 'สืบทอด' 'แทบ' 'วาทกรรม'
  'ความคิด' 'คสช.' 'ทั้งๆ ที่' 'ดำรงอยู่']
 ['ดีป้า' 'สำนักงานส่งเสริมเศรษฐกิจดิจิทัล' 'นิมมาน' 'ณัฐพล' 'depa' 'พัช'
  'บิ๊กแบง' 'ประยุกต์ใช้' 'เมือง' 'อัจฉริยะ' 'City' 'น่าอยู่' 'ไทยแลนด์'
  'กำลังคน' 'พิเชฐ' 'DEPA' 'ซิตี้' 'ภูเก็ต' 'ข้อมูลขนาดใหญ่' 'Cities'
  'โอที' 'คุณภาพชีวิต' 'การพัฒนา' 'ขอนแก่น' 'ประยุกต์' 'พื้นที่' 'ชุมชน'
  'เทคโนโลยี' 'Thailand' 'นวัตกรรม' 'อุบลราชธานี' 'Smart' 'จังหวัด'
  'ภาครัฐ' '

In [92]:
# BkkBiz: speed = 'learn' + n_gram vocab = True
top2vec(documents=bkkbiz_list, n_topics=10, speed='learn', ngram_vocab=True)

2022-11-16 09:41:36,766 - top2vec - INFO - Pre-processing documents for training
2022-11-16 09:43:35,664 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 09:58:50,130 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 09:58:56,000 - top2vec - INFO - Finding dense areas of documents
2022-11-16 09:58:59,373 - top2vec - INFO - Finding topics


Topic: [ 1  2  3  4  5  6  7  8  9 10]
Words: [['องค์กร' 'คอมพิวติ้ง' 'ไอที' 'คลาวด์' 'ออราเคิล' 'ไอ' '์ท' 'ซับซ้อน'
  'การทำงาน' 'ยืดหยุ่น' 'ฟอ' 'ความยืดหยุ่น' 'ทรานส์' 'ค็อก' 'เอ' 'ดจ์'
  'เลิร์น' 'ไฮบริด' 'เทคโนโลยี' 'ปัญญาประดิษฐ์' 'การประมวลผล' 'โซลูชั่น'
  'ร์' 'ไอบีเอ็ม' 'ทิฟ' 'แมชชีน' 'การเปลี่ยนแปลง' 'ความคาดหวัง'
  'ปรับเปลี่ยน' 'ชาญฉลาด' 'ตื่นตัว' 'สภาพแวดล้อม' 'ธิงส์(' 'คล่องตัว'
  'ธิงส์' 'สฟอร์' 'ชั่น' 'อัตโนมัติ' 'เคียว' 'โอที' 'เดลล์' 'พนักงาน'
  'ประมวลผล' 'ไล' 'เป็นตัว' 'เทรนด์' 'ซอฟต์แวร์' 'เผชิญ' 'นิง' 'ควอนตัม']
 ['เกาหลีเหนือ' 'ประธานาธิบดี' 'ซีเรีย' 'โอบามา' 'ผู้อพยพ' 'วอชิงตัน'
  'กรุง' 'ยูเอ็น' 'ปูติน' 'ผู้นำประเทศ' 'ผู้ต้องสงสัย' 'คิม' 'คว่ำบาตร'
  'โดนัลด์' 'การทูต' 'โซล' 'กองกำลัง' 'สังหาร' 'สันติภาพ' 'ผู้ก่อเหตุ'
  'ยูเครน' 'เรียกร้อง' 'ซาอุดีอาระเบีย' 'สิทธิมนุษยชน' 'แถลงการณ์'
  'ลิเบีย' 'ตุรกี' 'อิสลาม' 'กัวลาลัมเปอร์' 'ข่มขู่' 'ตอบโต้' 'ปักกิ่ง'
  'กระทรวงการต่างประเทศ' 'การก่อการร้าย' 'สำนักข่าว' 'ทางทหาร' 'ทวีต'
  'ทรัมป์' 'รัสเซีย' 'การตอบโต้' 'รอยเต

In [93]:
# Matichon: speed = 'learn' + n_gram vocab = True
top2vec(documents=matichon_list, n_topics=10, speed='learn', ngram_vocab=True)

2022-11-16 09:59:00,240 - top2vec - INFO - Pre-processing documents for training
2022-11-16 10:00:11,952 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 10:09:24,493 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 10:09:28,300 - top2vec - INFO - Finding dense areas of documents
2022-11-16 10:09:28,530 - top2vec - INFO - Finding topics


Topic: [ 1  2  3  4  5  6  7  8  9 10]
Words: [['ดำเนินคดี' 'ความผิด' 'กระทำความผิด' 'การจับกุม' 'เจ้าหน้าที่ตำรวจ'
  'การกระทำ' 'เข้าข่าย' 'จับกุม' 'พยานหลักฐาน' 'คำสั่งศาล' 'มีความผิด'
  'พนักงานสอบสวน' 'อาญา' 'บิดเบือน' 'ผู้ต้องหา' 'แจ้งความ' 'การปิดกั้น'
  'ผิดกฎหมาย' 'ประมวลกฎหมายอาญา' 'ข้อหา' 'มีโทษ' 'ผู้กระทำผิด' 'ปิดกั้น'
  'จำคุก' 'เท็จ' 'ผู้เสียหาย' 'โพสต์' 'ตามกฎหมาย' 'การสืบสวน' 'ศาล'
  'ปลุกปั่น' 'โทษ' 'สืบสวน' 'ตำรวจ' 'ท.' 'สอบสวน' 'เอาผิด' 'ควบคุมตัว'
  'หลบหนี' 'ทำผิด' 'คดี' 'ฝ่าฝืน' 'ออกหมายจับ' 'พ.ต.อ.' 'หลักฐาน' 'ระงับ'
  'ข้อกล่าวหา' 'ฟ้อง' 'หมายจับ' 'เบาะแส']
 ['รัฐประหาร' 'ประชาธิปไตย' 'ทางการเมือง' 'การเมือง' 'นักการเมือง'
  'การรัฐประหาร' 'รัฐธรรมนูญ' 'ยึดอำนาจ' 'อำนาจ' 'เผด็จการ'
  'พรรคอนาคตใหม่' 'การเลือกตั้ง' 'ร่างรัฐธรรมนูญ' 'อุดมการณ์' 'ระบอบ'
  'ตรงกันข้าม' 'การสืบทอด' 'ความขัดแย้ง' 'ม็อบ' 'ส.ว.' 'เลือกตั้ง'
  'ระบอบประชาธิปไตย' 'เรียกร้อง' 'ธร' 'พรรคการเมือง' 'ความชอบธรรม'
  'ประชามติ' 'ทักษิณ' 'ชุมนุม' 'บ้านเมือง' 'สืบทอด' 'พรรค'
  'การเคลื่อนไหว' 'ผู้ม

In [94]:
# BkkBiz: speed = 'deep-learn'
top2vec(documents=bkkbiz_list, n_topics=10, speed='deep-learn')

2022-11-16 10:09:43,877 - top2vec - INFO - Pre-processing documents for training
2022-11-16 10:11:39,292 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 12:43:32,805 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 12:43:39,245 - top2vec - INFO - Finding dense areas of documents
2022-11-16 12:43:42,770 - top2vec - INFO - Finding topics


Topic: [ 1  2  3  4  5  6  7  8  9 10]
Words: [['คอมพิวติ้ง' 'องค์กร' 'คลาวด์' 'ออราเคิล' 'ไอที' 'ยืดหยุ่น' 'ไอ' '์ท'
  'ค็อก' 'ฟอ' 'ทิฟ' 'ไฮบริด' 'ซับซ้อน' 'ความยืดหยุ่น' 'ดจ์' 'ทรานส์'
  'ไอบีเอ็ม' 'เลิร์น' 'โซลูชั่น' 'ธิงส์(' 'ธิงส์' 'แมชชีน' 'ร์' 'สฟอร์'
  'ความคาดหวัง' 'เอ' 'สภาพแวดล้อม' 'ปัญญาประดิษฐ์' 'กุญแจ' 'ไล' 'จำต้อง'
  'เดลล์' 'ซอฟต์แวร์' 'โอที' 'เคียว' 'แปซิฟิก' 'การทำงาน' 'ปฐ'
  'การประมวลผล' 'ตื่นตัว' 'เป็นตัว' 'นิง' 'การปฏิสัมพันธ์' 'ชาญฉลาด'
  'ไมโครซอฟท์' 'แง่มุม' 'คล่องตัว' 'ผู้ตอบ' 'ต้า' 'การคาดการณ์']
 ['โอบามา' 'เกาหลีเหนือ' 'ประธานาธิบดี' 'ซีเรีย' 'วอชิงตัน' 'ปูติน'
  'กรุง' 'คิม' 'โดนัลด์' 'ผู้อพยพ' 'ทำเนียบขาว' 'ตุรกี' 'ผู้ต้องสงสัย'
  'การทูต' 'ผู้นำประเทศ' 'ซาอุดีอาระเบีย' 'โซล' 'กองกำลัง' 'ยูเอ็น'
  'แถลงการณ์' 'คว่ำบาตร' 'สำนักข่าว' 'ก่อการร้าย' 'รัสเซีย' 'ผู้ก่อเหตุ'
  'ตอบโต้' 'ทรัมป์' 'สันติภาพ' 'เบอร์ลิน' 'กัวลาลัมเปอร์' 'สังหาร'
  'อิสลาม' 'ทวีต' 'ทางทหาร' 'จาการ์ตา' 'ลิเวอร์พูล' 'ปักกิ่ง' 'อิรัก'
  'หวด' 'ยูเครน' 'การก่อการร้าย' 'ลิเบีย' 'เยอรมนี' "ร

In [97]:
# Matichon: speed = 'deep-learn'
matichon_top2vec = top2vec(documents=matichon_list, speed='deep-learn')

2022-11-16 12:48:52,474 - top2vec - INFO - Pre-processing documents for training
2022-11-16 12:50:01,701 - top2vec - INFO - Creating joint document/word embedding
2022-11-16 14:42:06,881 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-16 14:42:11,321 - top2vec - INFO - Finding dense areas of documents
2022-11-16 14:42:11,594 - top2vec - INFO - Finding topics


Topic: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119]
Words: [['แบงก์กิ้ง' 'ธนาคาร' 'สินเชื่อ' ... 'พาณิชย์' 'Pay' 'ใช้จ่าย']
 ['ความผิด' 'กระทำความผิด' 'ดำเนินคดี' ... 'มาตรา' 'ฝ่าฝืน' 'อาชญากรรม']
 ['คลาวด์' 'องค์กร' 'โซลูชั่น' ... 'ลึก' 'ผู้ตอบ' 'เดลล์']
 ...
 ['บสย.' 'ค้ำประกัน' 'บรรษัท' ... 'เงินทุน' 'ผ่อนชำระ' 'กรุงไทย']
 ['ทริค' 'อิเล็ค' 'ไน' ... 'คาร์บอน' 'การไฟฟ้าส่วนภูมิภาค' 'PM']
 ['JKN' 'ซีรี่ส์' 'ออกอากาศ' ... 'มาน' 'กำไรสุทธิ' 'ลาว']]


In [ ]:
# Call documents from Top2Vec model
documents, document_scores, document_ids = top2vec.search_documents_by_topic(topic_num=0, 
                                                                             num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f'Document: {doc_id}, Score: {score}')
    print('-'*20)
    print(doc)
    print('-'*20)
    print()

### 2.5 BERTopic